# 12.2 넘파이처럼 텐서플로 사용하기

In [1]:
import tensorflow as tf
import numpy as np

## 12.2.1 텐서와 연산

In [2]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # 행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [11]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## 12.2.2 텐서와 넘파이

In [12]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [13]:
t.numpy() # 또는 np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [14]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [15]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## 12.2.3 타입 변환

In [16]:
tf.constant(2.) + tf.constant(40) # 에러 발생!!

InvalidArgumentError: ignored

In [17]:
tf.constant(2.) + tf.constant(40., dtype = tf.float64) # 에러 발생!!

InvalidArgumentError: ignored

In [18]:
t2 = tf.constant(40., dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 12.2.4 변수

In [19]:
v = tf.Variable([[1.,   2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [20]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [21]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [23]:
v.scatter_nd_update(indices = [[0, 0], [1, 2]], updates = [100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

# 12.3 사용자 정의 모델과 훈련 알고리즘

## 12.3.1 사용자 정의 손실 함수

In [24]:
def huber_fn(y_true, y_pred) :
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss   = tf.square(error) / 2
  linear_loss    = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

In [26]:
# 책에 없지만 코드 진행을 위해 추가
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test  = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state = 42)
X_train,      X_valid, y_train, y_valid     = train_test_split(X_train_full, y_train_full, random_state = 42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

input_shape = X_train.shape[1:]
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation = "selu", kernel_initializer = "lecun_normal", input_shape = input_shape),
                                 keras.layers.Dense(1),
])

In [27]:
model.compile(loss = huber_fn, optimizer = "nadam")

In [28]:
model.fit(X_train_scaled, y_train, epochs = 2, validation_data = (X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.5914 - val_loss: 0.2384
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2098 - val_loss: 0.2209


## 12.3.2 사용자 정의 요소를 가진 모델을 저장하고 로드하기

In [29]:
model.save("my_model_with_a_custom_loss.h5")

In [30]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5", 
                                custom_objects = {"huber_fn" : huber_fn})

In [31]:
def create_huber(threshold = 1.0) :
  def huber_fn(y_true, y_pred) :
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold

    squared_loss = tf.square(error) / 2
    linear_loss  = threshold * tf.abs(error) - threshold ** 2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  return huber_fn

model.compile(loss = create_huber(2.0), optimizer = "nadam")

In [32]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [33]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5", 
                                custom_objects = {"huber_fn" : create_huber(2.0)})

In [34]:
class HuberLoss(keras.losses.Loss) :
  def __init__(self, threshold = 1.0, **kwargs) :
    self.threshold = threshold
    super().__init__(**kwargs)

  def call(self, y_true, y_pred) :
    error = y_true - y_pred
    is_small_error = tf.abs(error) < self.threshold

    squared_loss = tf.square(error) / 2
    linear_loss  = self.threshold * tf.abs(error) - self.threshold ** 2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)

  def get_config(self) :
    base_config = super().get_config()
    return { **base_config, "threshold" : self.threshold }

In [35]:
model.compile(loss = HuberLoss(2.), optimizer = "nadam")

In [36]:
model.save("my_model_with_a_custom_loss_class.h5")

In [37]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5", custom_objects = {"HuberLoss" : HuberLoss} )

## 12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [38]:
def my_softplus(z) : # tf.nn.softplus(z)와 반환값이 같습니다.
  return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype = tf.float32) :
  stddev = tf.sqrt(2. / (shape[0] + shape[1]))
  return tf.random.normal(shape, stddev = stddev, dtype = dtype)

def my_l1_regularizer(weights) :
  return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights) : # tf.nn.relu(weights)와 반환값이 같습니다.
  return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [39]:
layer = keras.layers.Dense(30, activation = my_softplus,
                           kernel_initializer = my_glorot_initializer,
                           kernel_regularizer = my_l1_regularizer,
                           kernel_constraint  = my_positive_weights)

In [40]:
class MyL1Regularizer(keras.regularizers.Regularizer) :
  def __init__(self, factor) :
    self.factor = factor
  def __call__(self, weights) :
    return tf.reduce_sum(tf.abs(self.factor * weights))
  def get_config(self) :
    return { "factor": self.factor }

## 12.3.4 사용자 정의 지표

In [41]:
model.compile(loss = "mse", optimizer = "nadam", metrics = [create_huber(2.0)])

In [42]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [43]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [44]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [45]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [46]:
precision.reset_states # 두 변수가 0.0으로 리셋됩니다.

<bound method Precision.reset_states of <tensorflow.python.keras.metrics.Precision object at 0x7ff402fa1be0>>

In [47]:
class HuberMetric(keras.metrics.Metric) :
  def __init__(self, threshold = 1.0, **kwargs) :
    super().__init__(**kwargs) # 기본 매개변수 처리 (예를 들면, dtype)
    self.threshold = threshold
    self.huber_fn  = create_huber(threshold)
    self.total = self.add_weight("total", initializer = "zeros")
    self.count = self.add_weight("count", initializer = "zeros")

  def update_state(self, y_true, y_pred, sample_weight = None) :
    metric = self.huber_fn(y_true, y_pred)
    self.total.assign_add(tf.reduce_sum(metric))
    self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

  def result(self) :
    return self.total / self.count

  def get_config(self) :
    base_config = super().get_config()
    return { **base_config, "threshold" : self.threshold }

## 12.3.5 사용자 정의 층

In [49]:
class MyDense(keras.layers.Layer) :
  def __init__(self, units, activation = None, **kwargs) :
    super().__init__(**kwargs)
    self.units = units
    self.activation = keras.activations.get(activation)

  def build(self, batch_input_shape) :
    self.kernel = self.add_weight( 
        name = "kernel", shape = [batch_input_shape[-1], self.units],
        initializer = "glorot_normal")
    self.bias = self.add_weight(
        name = "bias", shape = [self.units], initializer = "zeros")
    super().build(batch_input_shape) # 마지막에 호출해야 합니다.

  def call(self, X) :
    return self.activation(X @ self.kernel + self.bias)

  def compute_output_shape(self, batch_input_shape) :
    return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
  
  def get_config(self) :
    base_config = super().get_config()
    return { **base_config, "units": self.units, 
            "activation": keras.activations.serialize(self.activation)}

In [50]:
class MyMultiLayer(keras.layers.Layer) :
  def call(self, X) :
    X1, X2 = X
    return [X1 + X2, X1 * X2, X1 / X2]

  def compute_output_shape(self, batch_input_shape) :
    b1, b2 = batch_input_shape

## 12.3.6 사용자 정의 모델

In [51]:
class ResidualBlock(keras.layers.Layer) :
  def __init__(self, n_layters, n_neurons, **kwargs) :
    super().__init__(**kwargs)
    self.hidden = [keras.layers.Dense(n_neurons, activation = "elu", kernel_initializer = "he_normal")
    for _ in range(n_layers)]

  def call(self, inputs) :
    Z = inputs
    for layers in self.hidden:
      Z = layer(Z)
    return inputs + Z

In [52]:
class ResidualRegressor(keras.Model) :
  def __init__(self, output_dim, **kwargs) :
    super().__init__(**kwargs)
    self.hidden1 = keras.layers.Dense(30, activation = "elu", 
                                      kernel_initializer = "he_normal")
    self.block1 = ResidualBlock(2, 30)
    self.block2 = ResidualBlock(2, 30)
    self.out = keras.layers.Dense(output_dim)

  def call(self, inputs) :
    Z = self.hidden1(inputs)
    for _ in range(1 + 3) :
      Z = self.block1(Z)

    Z = self.block2(Z)
    return self.out(Z)

## 12.3.7 모델 구성 요소에 기반한 손실과 지표

In [53]:
class ReconstructingRegressor(keras.Model) :
  def __init__(self, output_dim, **kwargs) :
    super().__init__(**kwargs)
    self.hidden = [keras.layers.Dense(30, activation = "selu", kernel_initializer = "lecun_normal")
    for _ in range(5)]
    self.out = keras.layers.Dense(output_dim)

  def build(self, batch_input_shape) :
    n_inputs == batch_input_shape[-1]
    self.reconstruct = keras.layers.Dense(n_inputs, name = 'recon')
    super().build(batch_input_shape)

  def call(self, inputs, training = None) :
    Z = inputs
    for layer in self.hidden :
      Z = layer(Z)
    
    reconstruction = self.reconstruct(Z)
    recon_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
    self.add_loss(0.05 * recon_loss)
    return self.out(Z)

## 12.3.8 자동 미분을 사용하여 그레이디언트 계산하기

In [54]:
def f(w1, w2) :
  return 3 * w1 ** 2 + 2 * w1 * w2

In [55]:
w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [56]:
(f(w1, w2 + eps) - f(w1, w2)) / eps

10.000000003174137

In [57]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
  z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])

In [58]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [59]:
with tf.GradientTape() as tape :
  z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1) # 36.0 텐서
dz_dw2 = tape.gradient(z, w2) # 실행 에러!!

RuntimeError: ignored

In [60]:
with tf.GradientTape(persistent = True) as tape:
  z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1) # 36.0 텐서
dz_dw2 = tape.gradient(z, w2) # 10.0 텐서, 작동 이상 없음!
del tape

In [61]:
dz_dw1, dz_dw2

(<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>)

In [62]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
  z = f(c1, c2)

  gradients = tape.gradient(z, [c1, c2]) # [None, None]이 반환

In [63]:
gradients

[None, None]

In [64]:
with tf.GradientTape() as tape:
  tape.watch(c1)
  tape.watch(c2)
  z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2]) # [36. 텐서, 10. 텐서] 반환
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [65]:
def f(w1, w2) :
  return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
  z = f(w1, w2) # stop_gradient() 없을 때와 결과가 같습니다.

gradients = tape.gradient(z, [w1, w2]) # => [30. 텐서, None] 반환
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [66]:
x = tf.Variable(100.)
with tf.GradientTape() as tape :
  z = my_softplus(x)

tape.gradient(z, [x])

[<tf.Tensor: shape=(), dtype=float32, numpy=nan>]

In [67]:
@tf.custom_gradient
def my_better_softplus(z) :
  exp = tf.exp(z)
  def my_softplus_gradients(grad) :
    return grad / (1 + 1 / exp)
  return tf.math.log(exp + 1), my_softplus_gradients

## 12.3.9 사용자 정의 훈련 반복

In [68]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation = "elu", kernel_initializer = "he_normal", 
                                                    kernel_regularizer = l2_reg),
                                 keras.layers.Dense(1, kernel_regularizer = l2_reg)
])

In [69]:
def random_batch(X, y, batch_size = 32) :
  idx = np.random.randint(len(X), size = batch_size)
  return X[idx], y[idx]

In [71]:
def print_status_bar(iteration, total, loss, metrics = None) :
  metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
  for m in [loss] + (metrics or [])])
  end = "" if iteration < total else "\n"

  print("\r{}/{} - ".format(iteration, total) + metrics, end = end)

In [72]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size

optimizer = keras.optimizers.Nadam(lr = 0.01)
loss_fn   = keras.losses.mean_squared_error

mean_loss = keras.metrics.Mean()
metrics   = [keras.metrics.MeanAbsoluteError()]

In [78]:
for epoch in range(1, n_epochs + 1) :
  print("에포크 {}/{}".format(epoch, n_epochs))

  for step in range(1, n_steps + 1) :
    X_batch, y_batch = random_batch(X_train_scaled, y_train)

    with tf.GradientTape() as tape:
      y_pred   = model(X_batch, training = True)
      main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
      loss      = tf.add_n([main_loss] + model.losses)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 494p 코드 추가 -----
    for variable in model.variables :
      if variable.constraint is not None :
        variable.assign(variable.constraint(variable))
    # -----

    mean_loss(loss)
    for metric in metrics :
      metric(y_batch, y_pred)
    print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)

  print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
  for metric in [mean_loss] + metrics:
    metric.reset_states()

에포크 1/5
11610/11610 - mean: 4475.5093 - mean_absolute_error: 0.8754
에포크 2/5
11610/11610 - mean: 3877.0715 - mean_absolute_error: 0.8119
에포크 3/5
11610/11610 - mean: 3434.7144 - mean_absolute_error: 0.8268
에포크 4/5
11610/11610 - mean: 2977.5823 - mean_absolute_error: 0.8116
에포크 5/5
11610/11610 - mean: 2517.7505 - mean_absolute_error: 0.7898


# 12.4 텐서플로 함수와 그래프

In [79]:
def cube(x) :
  return x ** 3

In [80]:
cube(2)

8

In [81]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [83]:
tf_cube = tf.function(cube)
tf_cube

In [84]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [85]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [86]:
@tf.function
def tf_cube(x) :
  return x ** 3

In [87]:
tf_cube.python_function(2)

8